In [2]:
(require [hy.contrib.walk [let]])

(let [x 1]
  (defn increment []
    (setv x (+ x 1))
    x))

(print (increment))
(print (increment))
(print (increment))

2
3
4


In [3]:
(import [urllib.request [Request urlopen]])

(defn get-raw-data-from-web [aUri &optional [anAgent {"User-Agent" "HyLangBook/1.0"}]]
  (setv req (Request aUri :headers anAgent))
  (setv httpResponse (urlopen req))
  (setv data (.read httpResponse))
  data)

(get-raw-data-from-web "http://markwatson.com")

b'<!DOCTYPE html>\n<html lang="en-us">\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    \n    <title>Mark Watson: AI Practitioner and Lisp Hacker | Mark Watson</title>\n    <meta name="viewport" content="width=device-width,minimum-scale=1">\n    <meta name="description" content="I am the author of 20&#43; books on Artificial Intelligence, Common Lisp, Deep Learning, Haskell, Java, Ruby, Hy language, and the Semantic Web. I have 55 US Patents.">\n    <meta name="generator" content="Hugo 0.78.0" />\n    \n    \n      <META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW">\n    \n\n    \n    \n      <link href="/dist/css/app.4fc0b62e4b82c997bb0041217cd6b979.css" rel="stylesheet">\n    \n\n    \n\n    \n      \n\n    \n\n    \n    \n      <link href="/index.xml" rel="alternate" type="application/rss+xml" title="Mark Watson" />\n      <link href="/index.xml" rel="feed" type="application/rss+xml" title="Mark Watson" />\n      \n    \n   

In [ ]:
;; This example was translated from the Python example in the Keras
;; documentation at: https://keras.io/examples/lstm_text_generation/

(import [keras.callbacks [LambdaCallback]])
(import [keras.models [Sequential]])
(import [keras.layers [Dense LSTM]])
(import [keras.optimizers [RMSprop]])
(import [keras.utils.data_utils [get_file]])
(import [numpy :as np]) ;; note the syntax for aliasing a module name
(import random sys io)

(setv path
      (get_file        ;; this saves a local copy in ~/.keras/datasets
        "nietzsche.txt"
        :origin "https://s3.amazonaws.com/text-datasets/nietzsche.txt"))

(with [f (io.open path :encoding "utf-8")]
  (setv text (.read f))) ;; note: sometimes we use (.lower text) to
                         ;;       convert text to all lower case
(print "corpus length:" (len text))

(setv chars (sorted (list (set text))))
(print "total chars (unique characters in input text):" (len chars))
(setv char_indices (dict (lfor i (enumerate chars) (, (last i) (first i)))))
(setv indices_char (dict (lfor i (enumerate chars) i)))

;; cut the text in semi-redundant sequences of maxlen characters
(setv maxlen 40)
(setv step 3) ;; when we sample text, slide sampling window 3 characters
(setv sentences (list))
(setv next_chars (list))

(print "Create sentences and next_chars data...")
(for [i (range 0 (- (len text) maxlen) step)]
  (.append sentences (cut text i (+ i maxlen)))
  (.append next_chars (get text (+ i maxlen))))

(print "Vectorization...")
(setv x (np.zeros [(len sentences) maxlen (len chars)] :dtype np.bool))
(setv y (np.zeros [(len sentences) (len chars)] :dtype np.bool))
(for [[i sentence] (lfor j (enumerate sentences) j)]
  (for [[t char] (lfor j (enumerate sentence) j)]
    (setv (get x i t (get char_indices char)) 1))
  (setv (get y i (get char_indices (get next_chars i))) 1))
(print "Done creating one-hot encoded training data.")

(print "Building model...")
(setv model (Sequential))
(.add model (LSTM 128 :input_shape [maxlen (len chars)]))
(.add model (Dense (len chars) :activation "softmax"))

(setv optimizer (RMSprop 0.01))
(.compile model :loss "categorical_crossentropy" :optimizer optimizer)

(defn sample [preds &optional [temperature 1.0]]
  (setv preds (.astype (np.array preds) "float64"))
  (setv preds (/ (np.log preds) temperature))
  (setv exp_preds (np.exp preds))
  (setv preds (/ exp_preds (np.sum exp_preds)))
  (setv probas (np.random.multinomial 1 preds 1))
  (np.argmax probas))

(defn on_epoch_end [epoch &optional not-used]
  (print)
  (print "----- Generating text after Epoch:" epoch)
  (setv start_index (random.randint 0 (- (len text) maxlen 1)))
  (for [diversity [0.2 0.5 1.0 1.2]]
    (print "----- diversity:" diversity)
    (setv generated "")
    (setv sentence (cut text start_index (+ start_index maxlen)))
    (setv generated (+ generated sentence))
    (print "----- Generating with seed:" sentence)
    (sys.stdout.write generated)
    (for [i (range 400)]
      (setv x_pred (np.zeros [1 maxlen (len chars)]))
      (for [[t char] (lfor j (enumerate sentence) j)]
        (setv (get x_pred 0 t (get char_indices char)) 1))
      (setv preds (first (model.predict x_pred :verbose 0)))
      (print "** preds=" preds)
      (setv next_index (sample preds diversity))
      (setv next_char (get indices_char next_index))
      (setv sentence (+ (cut sentence 1) next_char))
      (sys.stdout.write next_char)
      (sys.stdout.flush))
    (print)))

(setv print_callback (LambdaCallback :on_epoch_end on_epoch_end))

(model.fit x y :batch_size 128 :epochs 60 :callbacks [print_callback])

606208/600901 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 4us/step
corpus length: 600893
total chars (unique characters in input text): 84
Create sentences and next_chars data...
Vectorization...
Done creating one-hot encoded training data.
Building model...
Epoch 1/60
  40/1565 [..............................] - ETA: 0s - loss: 4.432 - ETA: 1:54 - loss: 3.971 - ETA: 2:26 - loss: 3.979 - ETA: 2:40 - loss: 3.868 - ETA: 2:45 - loss: 3.779 - ETA: 2:50 - loss: 3.686 - ETA: 2:59 - loss: 3.598 - ETA: 3:00 - loss: 3.555 - ETA: 3:02 - loss: 3.520 - ETA: 3:05 - loss: 3.470 - ETA: 3:10 - loss: 3.445 - ETA: 3:11 - loss: 3.413 - ETA: 3:11 - loss: 3.412 - ETA: 3:12 - loss: 3.393 - ETA: 3:12 - loss: 3.381 - ETA: 3:12 - loss: 3.367 - ETA: 3:11 - loss: 3.3